# CZSa 2024 Homework 1

## Instructions

You are allowed to collaborate and to use information from slides, books, internet etc., for solving this homework. HOWEVER, in the final exam, there will be some questions that are very similar to some questions of this homework and/or the next homework. These questions can give ten (10) points in total on the exam. Therefore, it is beneficial for you to understand the answers you provide here.

Deadline 2024-12-16 by email. Please include any code you have written either, separately or as verbatim text in the report. 

Please don't hestitate to ask if something is unclear.

## 1 (3p)
### a) 
Calculate numerically the SNR resulting from quantization of a real signal. For example, you can use the speech recording ```00011.wav``` available in the github repository. Plot the SNR vs the number of bits, e.g., for $\# \rm{bit}s=2,3,4,5,6,7,8$. \
You can use the functions ```quantize```, ```normalize``` and other code pieces from Exercise 1. 

### b) 
Compare the plot to the two results for SNR we derived in Lecture 1, i.e., 
\begin{equation}
SNR_{\rm{sine}}=6.02b + 1.67\,\,dB
\end{equation}
and
\begin{equation}
SNR_{\rm{uniform}}=6.02b\,\,dB
\end{equation}
where $SNR_{\rm{sine}}$ and $SNR_{\rm{uniform}}$ refers to the case of a sine signal and a uniformly distributed signal respectively (one example of a uniformly distributed signal is the sawtooth signal [https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.sawtooth.html](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.sawtooth.html)). In both formulas, $b$ is the number of bits used for the quantization. For both formulas, the error is assumed to be uniformly distributed in $[-q/2, q/2]$ where $q$ is the quantiztion step, i.e., the difference of two adjecent quantiztion levels. 

### c)
Analyse how accurate the assumptions for the above two formulas are for the signal you used. The assumptions about uniform distributions can be analaysed for example by plotting histograms. The assumptions about the signal being a pure sine or cosine signal can be checked by calculating DFT and plotting the spectrum.  

## 2 (2p)
Recall from Lecture 1 that if a signal is sampled with sampling frequency $F_s$ and before sampling contained only frequencies below $F_s/2$ then it can be reconstructed according to
\begin{equation}
f(t)=\sum_{n=-\infty}^{\infty} f\left(\frac{n}{F_s} \right )\frac{\sin(tF_s\pi -n\pi)}{tF_s\pi -n\pi}\quad (1),
\end{equation}
where $f\left(\frac{n}{F_s} \right )= f\left(nT_s \right )$ is the signal values at the sampling points ($F_s=1/T_s$).
### a)
Downsample (decimate) the signal used before with a factor 4 using the appropriate anti-aliasing filter. That is, if your original signal is in 16KHz (and accordingly has already been filtered so that it only contains frequencies below 8KHz) you

1. Create a low pass filter with cut off frequency 2KHz (you can use code snippets from exercise 4 based on ```scicpy.signal``` for this) and send the signal through the filter. Denote the obtained signal $x[n]$.
2. Discard 3 out of four samples, i.e., you keep every fourth sample of the signal. This can be written as $x_{\rm{d}}[n]=x[4n]$ where the left hand side is the downsampled signal and the right hand side is the original signal after anti-alias filtering. 

### b)
Use Equation (1) above to reconstruct some values of the original signal, $x[n]$, that are not part of $x_{\rm{d}}[n]$, e.g. $x[4n+2]$. Of of course, you need to replace the infinite limits in the sum with finite ones. It is a good idea to take equally many on each side of the point of interest. Try briefly how many you seem to need for accurate results. You don't need to analyse this deeply. 
Compare the result to the correct values as well as the ones obtained by simple linear interpolation of the two surrounding points, i.e., 
\begin{equation}
\frac{x_{\rm{d}}[n] + x_{\rm{d}}[n+1]}{2}=\frac{x[4n] + x[4(n+1)]}{2}
\end{equation}.

## 3 (2p)
Show that a symmetric FIR filter with with odd number of real coefficients has linear phase. Here, antisymmetric means the filter coefficients fulfill 
\begin{equation}
h[N-n]=h[N+n]
\end{equation}
where $N$ and $n$ are postitive integers, $n\leq N$ and the filter has length $2N+1$. (See slides of lecture 4 for a figure.)